# Segmentació d’Imatges amb U-Net

En aquesta pràctica treballarem amb U-Net, una de les arquitectures més utilitzades en tasques de segmentació d'imatges. L'objectiu principal és consolidar els coneixements teòrics adquirits, implementant i experimentant amb aquesta xarxa utilitzant un conjunt de dades artificial i simple.

La segmentació d'imatges és una tasca fonamental en visió per computador, ja que permet classificar cada píxel d'una imatge en una o més categories. En aquesta pràctica, el nostre objectiu serà entrenar una xarxa U-Net perquè pugui identificar regions específiques dins d’imatges generades artificialment. Aquest enfocament simplificat permetrà concentrar-nos en els aspectes clau de la implementació i el funcionament del model, sense les complicacions que podrien sorgir amb conjunts de dades més complexos.

El conjunt de dades que utilitzarem estarà format per imatges sintètiques que contenen formes geomètriques bàsiques (com cercles, quadrats o creus), amb les seves corresponents màscares que indiquen les àrees que cal segmentar. Això ens permetrà obtenir resultats visuals ràpidament i entendre millor com la xarxa aprèn a identificar patrons específics.

L'arquitectura que emprarem es pot utilitzar, per problemes més complexes, però amb les limitacions temporals que tenim no es recomenable.

In [1]:
from collections import OrderedDict
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import torch
import torch.nn as nn
import torch.optim as optim
from IPython import display
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm.auto import tqdm

## Dataset

Emprarem un dataset sintètic. Aquest originalment s'emprava per dur a terme tasques de Intel·ligència Artificial Explicable (XAI). Nosaltres ho emprarem per dur a terme una tasca de segmentació. Podeu trobar el dataset complet al següent [enllaç](https://github.com/miquelmn/aixi-dataset/releases/tag/1.5.0).

![Imatges dataset](https://ars.els-cdn.com/content/image/1-s2.0-S0004370224001152-gr001.jpg)

L'objectiu amb aquest dataset es segmentar del fons les formes geomètriques, emprant una U-Net. Nosaltres i a causa de les limitacions temporals que tenim i de recursos emprarem una versió [reduida](https://github.com/bmalcover/aa_2425/releases/download/v1/mini.zip).

In [4]:
# !wget https://github.com/miquelmn/aixi-dataset/releases/download/1.5.0/TXUXIv3.zip
!wget https://github.com/bmalcover/aa_2425/releases/download/v1/mini.zip
!unzip mini.zip

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


# Dataset de segmentació

A l’hora de treballar amb tasques de segmentació d’imatges, és essencial tenir un conjunt de dades que inclogui tant les imatges d’entrada com les màscares que representen les etiquetes de segmentació. Un dataset per segmentació ha de contenir:

1. Imatges d’entrada: Aquestes són les imatges que el model utilitzarà per aprendre. Poden estar en formats com JPEG o PNG.
2. Màscares de segmentació: Cada màscara és una imatge on cada píxel té un valor que representa la classe a la qual pertany (per exemple, 0 per fons, 1 per objecte). Les màscares han de tenir la mateixa mida que les imatges d’entrada.

In [2]:
class Formes(Dataset):
    """TXUXI segmentation dataset."""

    def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        path = self.paths[index]
        label = self.labels[index]

        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Depén de vosaltres
        label_img = cv2.imread(label, cv2.IMREAD_COLOR)
        label_img = cv2.add(label_img[:, :, 0],
                            cv2.add(label_img[:, :, 1], label_img[:, :, 2]))  # Depén de vosaltres

        image = self.transform(image)
        label_img = self.transform(label_img)

        if label_img.max() > 0:
            label_img = label_img / label_img.max()

        return image, label_img  # Gran diferència

Per instanciar aquest tipus de dataset es similar a com ho hem fet fins ara. **Per tal de simplificar l'entrenament només emprarem 500 mostres per l'entrenament i 200 de test**.

In [5]:
PATH_DADES = "./13_UNET/mini"  # POSA EL TEU!
# Dades entrenament
path_train = f"{PATH_DADES}/train"

img_files = sorted(glob(path_train + "/image/*.png"))
print(img_files)
label_files = sorted(glob(path_train + "/mask/*.png"))
img_files = img_files[:500]
label_files = label_files[:500]

print("total training images", len(img_files))

# Dades validacio

path_val = f"{PATH_DADES}/val"
img_files_val = sorted(glob(path_val + "/image/*.png"))
label_files_val = sorted(glob(path_val + "/mask/*.png"))
img_files_val = img_files_val[:200]
label_files_val = label_files_val[:200]

print("total test images", len(img_files_val))

[]
total training images 0
total test images 0


In [ ]:
train_batch_size = 4
test_batch_size = 4

# Definim una seqüència (composició) de transformacions
transform = transforms.Compose([
    transforms.ToTensor(),
    ## TODO: Put if necessary
])

train_data = Formes(img_files, label_files, transform)
val_data = Formes(img_files_val, label_files_val, transform)

train_loader = torch.utils.data.DataLoader(train_data, train_batch_size)
val_loader = torch.utils.data.DataLoader(val_data, test_batch_size)

## Entrenament
L’entrenament d’una xarxa U-Net consisteix a ensenyar al model a predir les màscares de segmentació corresponents a les imatges d’entrada. Aquest procés implica ajustar els pesos de la xarxa per minimitzar l’error entre les prediccions i les etiquetes reals (màscares). En aquesta secció, configurarem tot el necessari per entrenar el model amb el conjunt de dades generat prèviament.

Hi ha tot un conjunt de peculiaritats que fan que aquest entrenament difereixi respecte els vists fins ara:
- **Funció de pèrdua**. El Dice Coefficient és una mesura d’avaluació utilitzada en segmentació d’imatges per comparar la superposició entre la màscara predita i la màscara real. El seu valor oscil·la entre 0 (cap coincidència) i 1 (coincidència perfecta), i es calcula com el doble de la intersecció entre les dues màscares dividit per la seva suma total. També es pot emprar ``BCE``.
- **Sortida de la xarxa**. El tipus de problema determina el nombre de canals de sortida de la U-Net: per segmentació binària, la sortida és un únic canal amb valors que representen la probabilitat de pertànyer a la classe positiva (fons o objecte). En canvi, per segmentació multiclasse, la sortida té un canal per a cada classe i els valors representen la probabilitat de cada píxel de pertànyer a cadascuna de les classes.
- **Funcions d'activació**. Les funcions d'activació a la sortida d'una U-Net depenen del tipus de segmentació: en segmentació binària, s'utilitza una sigmoid per comprimir els valors entre 0 i 1, representant la probabilitat de pertànyer a la classe positiva. En segmentació, **sense superposició**, multiclasse, s'aplica una softmax per convertir els valors de cada canal en probabilitats normalitzades, assegurant que la suma sigui 1 per píxel.

### El model

In [ ]:
class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features

        ## CODER
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        ## DECODER

        self.upconv4 = nn.ConvTranspose2d(features * 16, features * 8, kernel_size=2,
                                          stride=2)  # Empra aquesta capa com exemple
        self.decoder4 = UNet._block(features * 16, features * 8, name="dec4")

        self.upconv3 = nn.ConvTranspose2d(features * 8, features * 4, kernel_size=2, stride=2)
        self.decoder3 = UNet._block(features * 8, features * 4, name="dec3")

        self.upconv2 = nn.ConvTranspose2d(features * 4, features * 2, kernel_size=2, stride=2)
        self.decoder2 = UNet._block(features * 4, features * 2, name="dec2")

        self.upconv1 = nn.ConvTranspose2d(features * 2, features, kernel_size=2, stride=2)
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.final = nn.Conv2d(
            in_channels=features,
            out_channels=out_channels,
            kernel_size=1,
            padding=0,
            bias=False,
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec1 = self.upconv4(bottleneck)
        dec1 = torch.cat((dec1, enc4), dim=1)
        dec2 = self.decoder4(dec1)

        dec2 = self.upconv3(dec2)
        dec2 = torch.cat((dec2, enc3), dim=1)
        dec3 = self.decoder3(dec2)

        dec3 = self.upconv2(dec3)
        dec3 = torch.cat((dec3, enc2), dim=1)
        dec4 = self.decoder2(dec3)

        dec4 = self.upconv1(dec4)
        dec4 = torch.cat((dec4, enc1), dim=1)
        dec5 = self.decoder1(dec4)
        return torch.sigmoid(self.final(dec5))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (name + "conv1",
                     nn.Conv2d(
                         in_channels=in_channels,
                         out_channels=features,
                         kernel_size=3,
                         padding=1,
                         bias=False,
                     ),
                     ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (name + "conv2",
                     nn.Conv2d(
                         in_channels=features,
                         out_channels=features,
                         kernel_size=3,
                         padding=1,
                         bias=False,
                     ),
                     ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

## La funció de pèrdua

Per fer tasques de segmentació, una de les funcions de pèrdua que podem emprar és el _Diceloss_ (intersecció vs unió):  El coeficient de _Dice_ s'utilitza habitualment en tasques de segmentació d'imatges com a mesura de la superposició entre les màscares de segmentació entre la predicció i el _ground truth_. El  _Diceloss_, és el complementari del coeficient de _Dice_, es pot utilitzar com a funció de pèrdua per entrenar models per a tasques de segmentació.

Dice Coefficient $= 2 \times \frac{|X \cap Y|}{|X| + |Y|}$



On:

- $X$ és la màscara de segmentació prevista.
- $Y$ és la màscara de segmentació de la veritat del sòl.
- $∣⋅∣$ denota la cardinalitat o el nombre d'elements d'un conjunt.

In [ ]:
class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 0.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
                y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

### Bucle d'entrenament

El bucle d'entrenament és un poc diferent al vist fins ara. En particular, en aquests moments els gràfics es van actualitzant a mesura que aprèn i així podem tenir un idea de cm va l'entrenament.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 40

model = UNet(1, 1).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = DiceLoss()


In [ ]:
t_loss = np.zeros(epochs)
v_loss = np.zeros(epochs)

pbar = tqdm(range(1, epochs + 1))  # tdqm permet tenir text dinàmic

for epoch in pbar:

    train_loss = 0
    val_loss = 0

    model.train()
    for batch_num, (input_img, target) in enumerate(train_loader, 1):
        input_img = input_img.to(device)
        target = target.to(device)

        output = model(input_img)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for input_img, target in val_loader:
            input_img = input_img.to(device)
            target = target.to(device)

            output = model(input_img)
            loss = criterion(output, target)
            val_loss += loss.item()

            # RESULTATS
    train_loss /= len(train_loader)
    t_loss[epoch - 1] = train_loss

    val_loss /= len(val_loader)
    v_loss[epoch - 1] = val_loss

    # VISUALITZACIO DINAMICA
    plt.figure(figsize=(12, 4))
    pl.plot(t_loss[:epoch], label="train")
    pl.plot(v_loss[:epoch], label="validation")
    pl.legend()
    pl.xlim(0, epochs)
    pl.xticks(range(0, epochs, 1), range(1, epochs + 1, 1))

    display.clear_output(wait=True)
    display.display(pl.gcf())
    plt.close()

    pbar.set_description(f"Epoch:{epoch} Training Loss:{train_loss} Validation Loss:{val_loss}")

In [ ]:
img, mask = next(iter(val_loader))
res = model(img.cuda()).detach().cpu().numpy()

plt.figure(figsize = (15, 10))
ax = plt.subplot(1, 3, 1)
ax.title.set_text('Imatge')
plt.imshow(img[1, 0, :, :]);
ax = plt.subplot(1, 3, 2)
ax.title.set_text('GT')
plt.imshow(mask[1, 0, :, :].detach().cpu().numpy());
ax = plt.subplot(1, 3, 3)
ax.title.set_text('Predicció')
plt.imshow(res[1, 0, :, :]);

# Tasques a fer

1. Que passa si canviam la funció de pèrdua i ara empram ``BCE``?
2. Fer un nou entrenament per segmentar de forma separada els diferents tipus de figura. Has d'adaptar el model. Quins canvis faries?